# Dataset Merge IDEA

1. history에 대해 drop duplicate 후 drop column      
2. watch 에 대해서도 drop duplicate 후 drop column     
3. 이후 inner join       
4. 다른 dataset merge   

문제 : 3번부터 램이 터져나감    
대안 1. 나눠서 조인 (노가다...)   
대안 2. 더 나은 노트북 환경 찾기   
대안 3. 똑똑하게 머리 쓰기 **<해결 방안 찾아냄>**

In [692]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

### 시청 시작 데이터에서 row 중복 체크 - 1만 6천개 가량의 중복 확인. Drop

In [693]:
history = pd.read_csv('history_dataset.csv', index_col=0)
history.drop_duplicates(subset=['profile_id','log_time','album_id'],inplace=True) 
# row = 1005651 rows × 8 columns
# after drop duplicates 899021 rows × 8 columns
# 106630 개 row 중복

In [694]:
data = history.copy()

In [695]:
history.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 899299 entries, 0 to 866336
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   profile_id       899299 non-null  int64  
 1   ss_id            899299 non-null  int64  
 2   log_time         899299 non-null  int64  
 3   act_target_dtl   899299 non-null  object 
 4   album_id         899299 non-null  int64  
 5   payment          59192 non-null   float64
 6   continuous_play  899252 non-null  object 
 7   short_trailer    899252 non-null  object 
dtypes: float64(1), int64(4), object(3)
memory usage: 61.7+ MB


# Meta Data merge 

In [696]:
# # 메타데이터 병합 
meta_df = pd.read_csv('../../data/meta_data.csv') # 42602 rows
meta_p = pd.read_csv('../../data/meta_data_plus.csv') # 767948 rows
# meta_merge = pd.merge(meta,meta_p,how='inner',on='album_id') #### meta 정보 최종 집합체 : 832356 rows
# data_w_meta = pd.merge(dataset,meta,how='inner',on='album_id')

/var/folders/wz/2dv1p69j3pv2c66bj9jt_hbc0000gn/T/ipykernel_57616/267488566.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_df = pd.read_csv('../../data/meta_data.csv') # 42602 rows


# profile data

In [697]:
profile_df = pd.read_csv('../../data/profile_data.csv')

# make data set

In [698]:
# 하이퍼파라미터 
class cfg: 
    gpu_idx = 0
    top_k = 25
    seed = 42
    neg_ratio = 5
    test_size = 0.1

In [699]:
data = data[['profile_id','album_id']]
data['rating'] = 1

In [700]:
cfg.n_users = data.profile_id.max()+1
cfg.n_items = data.album_id.max()+1

from sklearn.model_selection import train_test_split
from scipy import sparse
from tqdm.notebook import tqdm
# 학습 및 검증 데이터 분리
train, valid = train_test_split(
    data, test_size=cfg.test_size, random_state=cfg.seed,
)
print('학습 데이터 크기:', train.shape)
print('검증 데이터 크기:', valid.shape)

학습 데이터 크기: (809369, 3)
검증 데이터 크기: (89930, 3)


In [701]:
gt_valid = valid.copy()

In [702]:
# Matrix 형태로 변환 
train = train.to_numpy()
matrix = sparse.lil_matrix((cfg.n_users, cfg.n_items))
for (p, i, r) in tqdm(train):
    matrix[p, i] = r
    
train = sparse.csr_matrix(matrix)
train = train.toarray()
print("train 형태: \n", train)

  0%|          | 0/809369 [00:00<?, ?it/s]

train 형태: 
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [703]:
# profile_df['sex'] = profile_df['sex'].apply(lambda x: 0 if x =='F' else 1)
# 유저 특징 정보 추출 
profile_df = profile_df.set_index('profile_id')
user_features = profile_df[['age']].to_dict()
print("user_id 3의 age 정보 :", user_features['age'][3])

user_id 3의 age 정보 : 5


In [704]:
from sklearn.preprocessing import LabelEncoder

##### 아이템 특징 정보 추출 
meta_df = meta_df.set_index('album_id')

# 범주형 데이터를 수치형 데이터로 변경 
le = LabelEncoder()
meta_df['genre_mid'] = le.fit_transform(meta_df['genre_mid'])
item_features = meta_df[['genre_mid']].to_dict()
print("album_id 749의 genre_mid 정보 :", item_features['genre_mid'][749])

album_id 749의 genre_mid 정보 : 1


In [705]:
def make_UIdataset(train, neg_ratio):
    """ 유저별 학습에 필요한 딕셔너리 데이터 생성 
    Args:
        train : 유저-아이템의 상호작용을 담은 행렬 
            ex) 
                array([[0., 0., 0., ..., 0., 0., 0.],
                        [0., 0., 0., ..., 0., 0., 0.],
                        [0., 0., 0., ..., 0., 0., 0.],
                        ...,
                        [0., 0., 0., ..., 0., 0., 0.],
                        [0., 0., 0., ..., 0., 0., 0.],
                        [0., 0., 0., ..., 0., 0., 0.]])
        neg_ratio : negative sampling 활용할 비율 
            ex) 3 (positive label 1개당 negative label 3개)
    Returns: 
        UIdataset : 유저별 학습에 필요한 정보를 담은 딕셔너리 
            ex) {'사용자 ID': [[positive 샘플, negative 샘플], ... , [1, 1, 1, ..., 0, 0]]}
                >>> UIdataset[3]
                    [array([   16,    17,    18, ...,  9586, 18991,  9442]),
                    array([5, 5, 5, ..., 5, 5, 5]),
                    array([4, 4, 4, ..., 5, 1, 1]),
                    array([1., 1., 1., ..., 0., 0., 0.])]
    """
    UIdataset = {'profile_id': [], 'album_id':[],'age':[], 'genre_mid':[],'rating':[]}
    for user_id, items_by_user in tqdm(enumerate(train)):
#         UIdataset[user_id] = []
        # positive 샘플 계산 
        pos_item_ids = np.where(items_by_user > 0.5)[0]
        num_pos_samples = len(pos_item_ids)

        # negative 샘플 계산 (random negative sampling) 
        num_neg_samples = neg_ratio * num_pos_samples

        neg_items = np.where(items_by_user < 0.5)[0]
        neg_item_ids = np.random.choice(neg_items, min(num_neg_samples, len(neg_items)), replace=False)
        UIdataset['album_id'] += list(neg_item_ids)+list(pos_item_ids)
        UIdataset['profile_id'] += [user_id for _ in range(len(neg_item_ids)+len(pos_item_ids))]

        # feature 추출 
        features = []
        for item_id in np.concatenate([pos_item_ids, neg_item_ids]): 
            features.append(user_features['age'][user_id])
        UIdataset['age'] += features

        features = []
        for item_id in np.concatenate([pos_item_ids, neg_item_ids]): 
            features.append(item_features['genre_mid'][item_id])
        UIdataset['genre_mid'] += features

            # label 저장  
        pos_labels = np.ones(len(pos_item_ids))
        neg_labels = np.zeros(len(neg_item_ids))
        UIdataset['rating'] += list(neg_labels)+list(pos_labels)

    return UIdataset

In [706]:
pos_neg_samplings = make_UIdataset(train, 3)
train = pd.DataFrame(pos_neg_samplings)

0it [00:00, ?it/s]

In [707]:
train['profile_id'] = train['profile_id'].astype('category')
train['album_id'] = train['album_id'].astype('category')
train['genre_mid'] = train['genre_mid'].astype('category')

valid['genre_mid'] = valid['album_id'].apply(lambda x: item_features['genre_mid'][x])
valid['age'] = valid['profile_id'].apply(lambda x: user_features['age'][x])
valid['profile_id'] = valid['profile_id'].astype('category')
valid['album_id'] = valid['album_id'].astype('category')
valid['genre_mid'] = valid['genre_mid'].astype('category')

In [708]:
# meta_df = meta[['album_id','title','genre_mid','run_time']].drop_duplicates(subset=['album_id'], ignore_index=True)
# profile = pd.read_csv('../../data/profile_data.csv')
# profile_df = profile[['profile_id','sex','age']]

# dataset_meta_merge = pd.merge(pos_neg_samplings_df, meta_df, how='left', on='album_id')
# train = pd.merge(dataset_meta_merge,profile_df, how='left', on='profile_id')
qids_train = train.groupby('profile_id')['profile_id'].count().to_numpy()

In [709]:
pd.Series(qids_train).max(), pd.Series(qids_validation).max()

(5036, 857)

In [710]:
# valid_dataset_meta_merge = pd.merge(valid, meta_df, how='left', on='album_id')
# valid = pd.merge(valid_dataset_meta_merge,profile_df, how='left', on='profile_id')
qids_validation = valid.groupby('profile_id')['profile_id'].count().to_numpy()

# lightgbm rank

In [711]:
import numpy as np
import pandas as pd
import lightgbm

X_train = train.drop(columns=['rating'])
y_train = train['rating']

X_validation = valid.drop(columns=['rating'])
y_validation = valid['rating']

In [712]:
def recallk(actual, predicted, k = 25):
    """ label과 prediction 사이의 recall 평가 함수 
    Args:
        actual : 실제로 본 상품 리스트
        pred : 예측한 상품 리스트
        k : 상위 몇개의 데이터를 볼지 (ex : k=5 상위 5개의 상품만 봄)
    Returns: 
        recall_k : recall@k 
    """ 
    set_actual = set(actual)
    recall_k = len(set_actual & set(predicted[:k])) / min(k, len(set_actual))
    return recall_k

def unique(sequence):
    # preserves order
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

def ndcgk(actual, predicted, k = 25):
    set_actual = set(actual)
    idcg = sum([1.0 / np.log(i + 2) for i in range(min(k, len(set_actual)))])
    dcg = 0.0
    unique_predicted = unique(predicted[:k])
    for i, r in enumerate(unique_predicted):
        if r in set_actual:
            dcg += 1.0 / np.log(i + 2)
    ndcg_k = dcg / idcg
    return ndcg_k

def evaluation(gt, pred):
    """ label과 prediction 사이의 recall, coverage, competition metric 평가 함수 
    Args:
        gt : 데이터 프레임 형태의 정답 데이터 
        pred : 데이터 프레임 형태의 예측 데이터 
    Returns: 
        rets : recall, ndcg, coverage, competition metric 결과 
            ex) {'recall': 0.123024, 'ndcg': 056809, 'coverage': 0.017455, 'score': 0.106470}
    """    
    gt = gt.groupby('profile_id')['album_id'].unique().to_frame().reset_index()
    gt.columns = ['profile_id', 'actual_list']

    evaluated_data = pd.merge(pred, gt, how = 'left', on = 'profile_id')

    evaluated_data['Recall@25'] = evaluated_data.apply(lambda x: recallk(x.actual_list, x.predicted_list), axis=1)
    evaluated_data['NDCG@25'] = evaluated_data.apply(lambda x: ndcgk(x.actual_list, x.predicted_list), axis=1)

    recall = evaluated_data['Recall@25'].mean()
    ndcg = evaluated_data['NDCG@25'] .mean()
    coverage = (evaluated_data['predicted_list'].apply(lambda x: x[:cfg.top_k]).explode().nunique())/meta_df.index.nunique()

    score = 0.75*recall + 0.25*ndcg
    rets = {"recall" :recall, 
            "ndcg" :ndcg, 
            "coverage" :coverage, 
            "score" :score}
    return rets

In [713]:
model = lightgbm.LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    boosting_type="dart",
    n_estimators=1,
    importance_type='gain',
    verbose=1,
    random_state=42
)

In [714]:
model.fit(
    X=X_train,
    y=y_train,
    group=qids_train,
    eval_set=[(X_validation, y_validation)],
    eval_group=[qids_validation],
    eval_at=25,
    verbose=1,
)

/Users/krc/miniforge3/envs/krc/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/krc/miniforge3/envs/krc/lib/python3.8/site-packages/lightgbm/basic.py:1780: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/krc/miniforge3/envs/krc/lib/python3.8/site-packages/lightgbm/basic.py:1513: UserWarning: categorical_column in param dict is overridden.
  _log_warning(f'{cat_alias} in param dict is overridden.')


[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007997 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 31346
[LightGBM] [Info] Number of data points in the train set: 1426664, number of used features: 4
[1]	valid_0's ndcg@25: 1


LGBMRanker(boosting_type='dart', importance_type='gain', metric='ndcg',
           n_estimators=1, objective='lambdarank', random_state=42, verbose=1)

In [715]:
def valid_epoch(cfg, model, data):
    pred_list = []

    query_user_ids = data['profile_id'].unique() # 추론할 모든 user array 집합
    full_item_ids = np.array([c for c in range(cfg.n_items)]) # 추론할 모든 item array 집합 
    # full_item_ids_feat1 = [user_features['sex'][c] for c in query_user_ids]
    full_item_ids_feat2 = [item_features['genre_mid'][c] for c in full_item_ids]

    for user_id in tqdm(query_user_ids):

        input_data = pd.DataFrame()
        input_data['album_id'] = list(full_item_ids)
        input_data['profile_id'] = user_id
        input_data['age'] = user_features['age'][user_id]
        input_data['genre_mid'] = full_item_ids_feat2

        input_data['album_id'] = input_data['album_id'].astype('category')
        input_data['profile_id'] = input_data['profile_id'].astype('category')
        input_data['genre_mid'] = input_data['genre_mid'].astype('category')

        pred_u_score = model.predict(input_data)

        pred_u_idx = np.argsort(pred_u_score)[::-1]
        pred_u = full_item_ids[pred_u_idx]
        pred_list.append(list(pred_u[:cfg.top_k]))

    pred = pd.DataFrame()
    pred['profile_id'] = query_user_ids
    pred['predicted_list'] = pred_list
    return pred

In [716]:
pred = valid_epoch(cfg, model, X_validation)
rets = evaluation(gt_valid, pred)

  0%|          | 0/6665 [00:00<?, ?it/s]

In [717]:
rets

{'recall': 0.00059620643050324,
 'ndcg': 0.00034833145286859154,
 'coverage': 0.005065830721003135,
 'score': 0.0005342376860945779}

In [718]:
submission = pd.read_csv('../../data/sample_submission.csv')
# submission = valid_epoch(cfg, model, submission, mode='test')

In [719]:
submission = valid_epoch(cfg, model, submission)

  0%|          | 0/8311 [00:00<?, ?it/s]

In [720]:
submission.head(60)

,profile_id,predicted_list
0,3,"[12958, 18072, 7112, 7113, 7114, 7115, 7117, 1..."
1,5,"[0, 9059, 9039, 9040, 9041, 9042, 9043, 9044, ..."
2,7,"[12958, 18072, 7112, 7113, 7114, 7115, 7117, 1..."
3,12,"[12958, 18072, 7112, 7113, 7114, 7115, 7117, 1..."
4,16,"[12958, 19213, 19211, 19210, 19209, 19208, 192..."
5,19,"[12958, 19213, 19211, 19210, 19209, 19208, 192..."
6,20,"[12958, 18072, 7112, 7113, 7114, 7115, 7117, 1..."
7,22,"[12958, 5621, 19215, 19214, 19213, 19212, 1921..."
8,24,"[12958, 18072, 7112, 7113, 7114, 7115, 7117, 1..."
9,26,"[12958, 18072, 7112, 7113, 7114, 7115, 7117, 1..."


In [721]:
submission.to_csv('15est_lgbmrank_submission.csv', index = False)